In [5]:

from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

import math
import random

import numpy as np


gpu up


In [2]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [3]:
threshold = 150  # for Binarize


def Binarize(X):
    X[X < threshold] = 0
    X[X >= threshold] = 1
    return X


# train_X = Binarize(train_X)
# test_X = Binarize(test_X)

def Normalize(X):
    return X/255

# train_X = Normalize(train_X)
# test_X = Normalize(test_X)


In [4]:
def centroid(arr):
    moments = {'M00': arr.sum(axis=1).sum(),
               'M01': (np.arange(start=1, stop=arr.shape[1] + 1, step=1) * arr.sum(axis=0).reshape(1, -1)).sum(),
               'M10': (np.arange(start=1, stop=arr.shape[0] + 1, step=1) * arr.sum(axis=1).reshape(1, -1)).sum(),
               'M11': 0}

    x = moments['M10'] / moments['M00'] if moments['M00'] != 0 else 0
    y = moments['M01'] / moments['M00'] if moments['M00'] != 0 else 0
    return [x, y]


In [5]:
def imageWin(trainx, r, c):
    temp = []
    for i in range(0, int(trainx.shape[0]), r):
        for j in range(0, int(trainx.shape[1]), c):
            temp.append(centroid(trainx[i:i+r, j:j+c]))
    return np.array(temp)


def slicingArray(trainX, r, c):
    featureVector = []
    for trainx in trainX:
        featureVector.append(imageWin(trainx, r, c))
    return np.array(featureVector)


In [6]:
## Shuffling the dataset

train_X, train_y = shuffle(train_X[:15000], train_y[:15000])
test_X, test_y = shuffle(test_X[:1000], test_y[:1000])

train_y =  (np.eye(10)[np.array(train_y)]).tolist()
test_y = (np.eye(10)[np.array(test_y)]).tolist()

arr = slicingArray(train_X, 4, 7)
# trainFeatureVec = arr.reshape([*arr.shape[:-2], -1])

trainFeatureVec = arr.tolist()

# print(trainFeatureVec.shape)

arr = slicingArray(test_X, 4, 7)
# testFeatureVec = arr.reshape(*arr.shape[:-2], -1)
testFeatureVec = arr.tolist()

In [104]:
# Some Fundamental Functions
def listXlist(l1: list, l2:list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l1))]
    # print(len(l1),len(l1[0])," ",len(l2),len(l2[0]))
    for i in range(len(l1)):
        for j in range(len(l2[0])):
            for k in range(len(l2)):
                res[i][j] += l1[i][k] * l2[k][j]
    return res

def listMINUSlist(l1: list, l2:list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l2))]
    for i in range(len(l2)):
        for j in range(len(l1[0])):
                res[i][j] = l1[i][j] - l2[i][j]
    return res

def listSquare(l: list):
    res = []
    for i in range(len(l)):
        res.append([l[i]**2])
    return res

def mean(k: list):
    return sum([i[0] for i in k]) / len(k)

In [173]:
# Fully connected NN
class NN:
    def __init__(self, m: int, n: list, activation: str, e: int, lr: float, outShape: int):
        self.neurons = n
        self.neuronsListBefore = []
        self.neuronsListAfter = []
        self.layers = m
        self.weights = dict([(x, [0]) for x in range(self.layers + 1)])
        self.grads = dict([(x, [0]) for x in range(self.layers + 1)])
        # self.bias = np.random.random()
        self.activationFn = activation
        self.learning_rate = lr
        self.epochs = e
        self.outputShape = outShape

    # MSE
    def __costFunc(self, y, preds):
        return 0.5 * listSquare(listMINUSlist(y, preds))

    def __activation(self, x: float, type: str):
        res = {
            'sigmoid': 1 / (1 + math.exp(-x)),
            'tanh': math.tanh(x),
            'relu': max(x, 0),
            'linear': x,
            'leaky_relu': max(0.1 * x, x),
        }
        return res[type]

    def __derivatives(self, out: float, type: str):
        res = {
            'sigmoid': out * (1 - out),
            'tanh': 1 - (out ** 2),
            'relu': 0 if out < 0 else 1,
            'linear': 1,
            'leaky_relu': 0.1 if out < 0 else 1,
        }
        return res[type]

    def __initializeWeights(self, input: list, outputShape: int):
        self.weights[0] = [[random.random() for i in range(len(input))] for j in range(self.neurons[0])]
        self.grads[0] = [[0 for i in range(len(input))] for j in range(self.neurons[0])]

        for i in range(1, self.layers):
            self.weights[i] = [[random.random() for i in range(self.neurons[i - 1])] for j in range(self.neurons[i])]
            self.grads[i] = [[ 0 for i in range(self.neurons[i - 1])] for j in range(self.neurons[i])]

        self.weights[self.layers] = [[random.random() for i in range(self.neurons[self.layers - 1])  ] for j in range(outputShape)]
        self.grads[self.layers] = [[0 for i in range(self.neurons[self.layers - 1])  ] for j in range(outputShape)]
    
    def __zero_grad(self, input: list, outputShape: int):
        self.grads = dict([(x, [0]) for x in range(self.layers + 1)])
        self.grads[0] = [[0 for i in range(len(input))] for j in range(self.neurons[0])]

        for i in range(1, self.layers):
            self.grads[i] = [[ 0 for i in range(self.neurons[i - 1])] for j in range(self.neurons[i])]

        self.grads[self.layers] = [[0 for i in range(self.neurons[self.layers - 1])  ] for j in range(outputShape)]


    # def fit(self, trainX: list, trainY: np.array):
    #     self.__initializeWeights(trainX, self.outputShape)
    #     self.neuronsListAfter.append(trainX)
    #     for epoch in range(self.epochs):
    #         outputLayer = self.__propagate(trainX)
    #         self.__backPropagation(trainY, outputLayer)
    #         self.updateWeights()

    def fit(self, trainX: list, trainY: list):
        self.__initializeWeights(trainX[0], self.outputShape)
        for epoch in range(self.epochs):
            for i in  range(len(trainX)):
                self.neuronsListBefore = []
                self.neuronsListAfter = []    
                outputLayer = self.__propagate(trainX[i])
                self.__backPropagation(trainY[i], outputLayer)
                self.updateWeights()
                self.__zero_grad(trainX[i], self.outputShape)


    def __feedForward(self, input: list, layerIdx: int) -> list:
        out = []
        x = []
        # print(input,self.weights[layerIdx]) 
        # print(layerIdx)
        x = listXlist(self.weights[layerIdx], input)
        self.neuronsListBefore.append(x)
        for i in range(len(x)):
            out.append([self.__activation(x[i][0], self.activationFn)])

        self.neuronsListAfter.append(out)
        return out


    def __propagate(self, x: list):
        self.neuronsListAfter.append(x)
        out = self.__feedForward(x, 0)
        for i in range(1, self.layers + 1):
            out = self.__feedForward(out, i)
        return out


    def __backPropagation(self, target: list, outputLayer: list):
        # print(len(outputLayer),outputLayer)
        dw = -1 * mean(listMINUSlist(target, outputLayer))
        deltas = dict([(x, [0]) for x in range(self.layers + 1)])
        deltas[self.layers] = np.array([k[0] for k in listMINUSlist(target, outputLayer)])/-10 
        for i in range(len(self.weights[self.layers])):
            deltas[self.layers][i] *= self.__derivatives(self.neuronsListBefore[self.layers][i][0], self.activationFn)
            for j in range(len(self.weights[self.layers][0])):
                # print(self.neuronsListAfter[self.layers - 1])
                self.grads[self.layers][i][j] = deltas[self.layers][i] * self.neuronsListAfter[self.layers][j][0] 

        # [sum([self.weights[j][i] * delta[layer + 1][j] for j in range(len(delta[layer]))]) for i in range(len(weights))]

        for layer in reversed(range(0,self.layers)):
            deltas[layer] = [sum([self.weights[layer + 1][j][i] * deltas[layer + 1][j] for j in range(len(deltas[layer]))]) for i in range(len(self.weights[layer + 1][0]))]
            for i in range(len(self.weights[layer])):
                deltas[layer][i] *= self.__derivatives(self.neuronsListBefore[layer][i][0], self.activationFn)
                # print(deltas)
                for j in range(len(self.weights[layer][0])):
                    # print(layer, i,j)
                    self.grads[layer][i][j] = deltas[layer][i] * self.neuronsListAfter[layer][j][0]


    def updateWeights(self):
        for layer in range(self.layers + 1):
            for i in range(len(self.weights[layer])):
                for j in range(len(self.weights[layer][0])):
                    self.weights[layer][i][j] -= (self.grads[layer][i][j] * self.learning_rate)


    def predict(self, testX: list):
        preds = []
        for i in range(len(testX)):
            preds.append(self.__propagate(testX[i]))
        return preds


In [90]:
list(reversed(range(0,nn.layers)))

[1, 0]

In [83]:
nn.neuronsListAfter

[[[2], [3], [4]],
 [[0.9727722388458349],
  [0.9992961981162238],
  [0.8927446163103013],
  [0.9986772569448514]],
 [[0.906939361013108],
  [0.7900422311533066],
  [0.8779316096631612],
  [0.9163678087156085]],
 [[0.73807081220105], [0.812315090682068], [0.841127632254201]]]

In [99]:
nn.neuronsListAfter[0]

[[2], [3], [4]]

In [103]:
len([[[2],[3],[4]]])

1

In [185]:
nn.predict(X[:6])

[[[0.0]],
 [[7.000000000000001]],
 [[14.000000000000002]],
 [[21.0]],
 [[28.000000000000004]],
 [[35.0]]]

In [183]:
y[:6]

[[[0]], [[7]], [[14]], [[21]], [[28]], [[35]]]

In [182]:
X = [ [[i]] for i in range(50)]
y= [ [[d[0][0]*7]] for d in X]

In [184]:
nn = NN(2, [2,2],'linear', 100, 0.0003, 1)
nn.fit(X, y)